In [ ]:
#!/usr/bin/env python3

import os
import sys
from tqdm import trange,tqdm
import threading
from scipy.optimize import differential_evolution

def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

persons = {}
skills = {}
projects = {}

def get_skilled_persons(skill):
    if skill not in skills:
        skills[skill] = []
    return skills[skill]

class Person:
    def __init__(self):
        self.name, nb_skills = input().split()
        nb_skills = int(nb_skills)
        self.skills = {}
        for _ in range(nb_skills):
            skill,level = input().split()
            self.skills[skill] = int(level)
            get_skilled_persons(skill).append(self)
        
        self.taken_until = 0

    def get_level(self, skill):
        if skill in self.skills:
            return self.skills[skill]
        return 0

    def get_nb_skills(self):
        return len(self.skills)


class Project:
    def __init__(self):
        self.name,d,s,b,r = input().split()
        self.duration = int(d)
        self.reward = int(s)
        self.deadline = int(b)
        nb_role = int(r)
        self.needed_skills = []
        for _ in range(nb_role):
            skill,level = input().split()
            self.needed_skills.append((skill,int(level)))

nb_person,nb_project = input().split()
nb_person = int(nb_person)
nb_project = int(nb_project)

eprint("## Read persons")

for _ in trange(nb_person):
    p = Person()
    persons[p.name] = p

eprint("## Read projects")

for _ in trange(nb_project):
    p = Project()
    projects[p.name] = p


# Sort ascending skills
for skill,skilled in skills.items():
    skilled.sort(key=lambda p: p.get_level(skill), reverse=True)


# Planning

def project_priority(p: Project):
    return p.deadline

ordered_projects = sorted(projects.values(), key=project_priority)

needed_skills = []
for p in ordered_projects:
    needed_skills.extend(p.needed_skills)

# Borne sup
def bound_for_skill(skill, level):
    skilled = get_skilled_persons(skill)
    i = 1
    while skilled[i].get_level(skill) >= level:
        i+=1
    return i



# la fitness
def compute_score(x):
    x = [int(i) for i in x]
    projects = ordered_projects.copy()
    aviability = {p: 0 for p in persons}
    day = 0
    score = 0

    teams = {}
    i = 0
    for proj in projects:
        t = []
        teams[proj.name] = t
        for skill,_ in proj.needed_skills:
            t.append(get_skilled_persons(skill)[x[i]])
            i+=1

    def is_ready(proj):
        for pers in teams[proj.name]:
            if aviability[pers] > day:
                return False
        return True

    def pick_project():
        for i,proj in enumerate(projects):
            if is_ready(proj):
                return projects.pop(i)

    while len(projects) > 0:
        proj = pick_project()

        while proj != None:
            for pers in teams[proj.name]:
                aviability[pers] = day + proj.duration

            score += max(0, proj.reward - max(0, day + proj.duration + 1 - proj.deadline))

            proj = pick_project()

        day += 1



    return -score # differential_evolution cherche le min

eprint("# Let darwin do its job")

res = differential_evolution(
    compute_score,
    bounds=[(0, bound_for_skill(*sk)) for sk in needed_skills],
    # strategy
    #maxiter
)

eprint("# Printing results")

# Print results
print(len(ordered_projects))
i = 0
for proj in ordered_projects:
    print(proj.name)
    t = []
    for skill,_ in proj.needed_skills:
        t.append(get_skilled_persons(skill)[res.x[i]])
        i+=1
    print(*t)